# 🧠 LangChain RAG Chatbot (Google Colab Compatible)
This notebook allows you to upload `.txt`, `.csv`, or `.pdf` files using Colab, chunk them with LangChain, embed using HuggingFace, and query using a RetrievalQA chain.

In [1]:
!pip install langchain langchain_community faiss-cpu openai tiktoken chromadb unstructured sentence-transformers

In [2]:
from google.colab import files
from langchain_community.document_loaders import TextLoader, CSVLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.chat_models import ChatOpenAI

In [3]:
# Upload your document
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load document based on file type
if filename.endswith(".txt"):
    documents = TextLoader(filename).load()
elif filename.endswith(".csv"):
    documents = CSVLoader(file_path=filename).load()
elif filename.endswith(".pdf"):
    documents = PyPDFLoader(filename).load()
else:
    raise ValueError("Unsupported file format")

Saving 406.-Generative-AI-Guide_ver1-EN.pdf to 406.-Generative-AI-Guide_ver1-EN.pdf


In [4]:
# Split document into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

In [5]:
# Create vectorstore with embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

<ipython-input-5-347f04a34d50>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# Build RetrievalQA chain

from langchain_community.chat_models import ChatOpenAI

api_key = ""  # 🔒 REPLACE with your real key
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=api_key)

qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)


# Ask a question
#response = qa.run("What is this document about?")
response = qa.invoke({"query": "What is this document about?"})

print(response)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}